# Session 1

# Session 2

# Session 3

# Session 4

# Session 5

# Session 6

# Session 7

# Session 8

# Session 9

# Session 10

# Session 11

# Session 12 - Synthesis - Overview
first, lets review the whole codesign flow

## Specification
starting point of codesign, high-level system specification (functional and behavioral)

## Synthesis
#### Stages
1. Allocation: chosing system's sections
2. Partitioning: mapping model's sections to system's sections
3. Scheduling: scheduling execution of sections

#### Analysis and Evaluation
at each stage, we need to evaluate the system by simulation or analysis
- functional
- quality measures

### Allocation
- the beginning of design space exploration.
- specifying sections (processor, memory, special sections), their quantity and their connections.
- done by the designer.

### Partitioning
- mapping behavioural sections specified in specification stage to the sections specified in allocation's architecture
- trying to respect system limitations and keep the quality at the same time.
- several algorithms have been studied to help this stage.
- system specification may be modified while doing this stage.

### Scheduling
- respecting system limitations
- respecting dependencies between sections
- reducing syncronization overhead resulted by messaging

Strategies:
1. Dynamic scheduling
    - when data flow is determined in **runtime**
    - minimum scheduling
2. Static scheduling
    - execution times and conditional branches are estimated
    - bad estimations leads to expensive syncronization overheads

## Backend
implementation: the result of synthesis will be given to several tools
1. processor => compiler => machine code
2. ASIC => highlevel synthesis tools => PCB
3. connections => like ASIC

# Session 13 - Synthesis - Partitioning - Algorithms

## Partitioning methods
Flow:
1. transform functional requirements to processes
2. estimate the execution time of each process on processing elements

### Design Space Exploration
explore states and choose the best answer which meets all requirements.

1. exhaustive search (brute force): costly
2. exact methods: based on mathematical modeling, suitable for small design spaces
3. heuristic methods
4. meta heuristic methods (e.g evolutional algorithms)

- inputs: task graph (similar to DFG), system architecture (sections and connections)
- output: mapping processes in task graph to section in architecture

categorizing strategies based on their priority regarding the limitations (performance, cost):
- primal approach - performance based: Vulcan
- dual approach - cost based: Cosyma

# Session 14 - Synthesis - Partitioning - Algotithms - Vulcan

## Vulcan algorithm
efficiency based => hardware based => balancing cost by iterative pruning of unnecessary concurrencies and mapping those processes to software

language: HardwareC
- compiled into a flow graph similar to task graph
- each node is a low-level operation
- each edge is a binary condition
    - controls concurrency
    - controls execution time of each node

task graph transforms into threads (e.g on wait nodes) => threads map to CPU ($\phi _S$) or ASIC ($\phi _H$)

cost function:
$$f(\omega)=c_1S_h(\phi_H)-c_2S_s(\phi _S)+c_3B-c_4P+c_5|m|$$
in which:
- $S_h$ is HW size
- $S_s$ is SW size
- $B$ is bus transfer rate
- $P$ is processor transfer rate
- $|m|$ is number of variables
- $c_1 ... c_5$ are constants

stages:
1. initial partition: mapping all threads to $\phi _H$
2. looping:
    1. candidating some threads to be mapped into $\phi _S$
        1. calculating performance and continuing if requirements are met
            - calculation is done by estimating the worst case delay caused by the new partitioning in task graph
    2. updating cost function to evaluate effectiveness of the new partitioning
    
to minimize communication cost, with each thread transportation from $\phi _H$ to $\phi _S$ the very next node to the thread will be candidated.

there's no step-back in transportation (from $\phi _S$ to $\phi _H$)

# Session 14 - Synthesis - Partitioning - Algorithms - Cosyma and GCLP

## Cosyma algorithm
language: $C^x$, compiled into a CDFG

code is quantized into "basic blocks", which are branch-free blocks. these blocks are then partitioned into HW and SW

1. all basic blocks are assigned to CPU
2. loop:
    1. moving each basic block to ASIC and calculating the change in cost (execution time):
    $$\Delta c(b)=\omega(t_H(b)-t_S(b)+\Delta t_{com}(b))\times It(b)$$
        in which:
        - $\Delta c$ is the change of the cost function caused by moving the basic block $b$
        - $\omega$ is constant
        - $t_H(b)$ is execution time of basic block $b$ in HW
        - $t_S(b)$ is execution time of basic block $b$ in SW
        - $\Delta t_{com}(b)$ is estimated change in communtication time
        - $It(b)$ is total number of times $b$ is executed
    2. if the change was negative then the block is moved, else, the block is moved with a probability which decreases with each iteration (simulated annealing)
    3. another block is selected in neighbourhood

## Global Critically/Local Phase algorithm (GCLP)
a more balanced approach
### global critically (performance based)
a measure based on critically of system latency. the more critical system the more HW mapping. re-computed in every iteration based on already scheduled nodes and remaining latency capacity.
### local phase (cost based)
a per node value, calculated based on 3 phases each node can be categorized into. 

### objective functions
each node is mapped either to SW or HW based on comparison made between GC and almost LP.

# Session 16 - Synthesis - Partitioning - Algorithms - Distributed Systems

## Heuristic methods

### iterative methods
1. random initial state
2. search in neighbourhood and choose a candidate state
3. evaluate the candidate according to the cost function and system requirements
    - greedy approach (local minimum problem)
    - other complex approaches (e.g simulated annealing)

### constructive methods
- random partitioning
- hierarchial categorization

## Distributed Systems
multiprocessor architecture
- less focus on ASIC design and architecture
- non-homogeneous structure
    - more usage
    - complex and diverse in:
        - processing elements
        - processing power
- commonly used (MPSoC)

Methods:
- exact methods (e.g modeling based on Integer Linear Programming)
- heuristic methods (e.g wolf algorithm)

### wolf method
pros:
- faster
- optimized and performant solution for big problems

stages:
1. initial state: mapping each process to processing elements according to their timing order (scheduling)
2. remapping based on costs
    - first, processes on processing elements with lesser utilization rate are moved to SW. another option is removing these processing elements or replacing them with more suitable elements
    - processing elements may merge and load is balanced
3. remapping based on communication cost
    - maximizing inter-block communication
4. mapping communication channels based on results
5. mappnig IO utils to processing elements

# Session 17 - Synthesis - Partitioning - Algorithms - Distributed Systems - ILP

## Exact Methods
mathematical modeling which leads to the most optimized solution
- NP-complete => exponential growth of time complexity
- solvable only in relaxed situations by solver tools

### integer linear programming
linear programming types:
1. linear programming (LP)
2. integer linear programming (ILP)
3. mixed integer linear programming (MILP)

system variables:
$$X=\{x_1,x_2,x_3,...,x_n\}$$

objective function:
$$C(X)=\sum_{x_i \in X} a_ix_i$$

constraints:
$$\forall j \in J:\sum_{x_i \in X} b_{i,j}x_i \geq c_j$$

problem is minimizing/maximizing the objective function while constraints are regarded.

### Prakash and Parker method
based on ILP

steps:
1. defining variables
2. defining objective function based on costs of elements and connections
3. defining contraints (also partitioning constraints: one processor per process, chronology, preventing overcrowd, ...)

- best answer
- task graph with maximum 9 nodes
- perspective towards effectiveness of heuristic methods (suboptimal states)